In [2]:
import os
import shutil
import random
import math
import numpy as np
import matplotlib.pyplot as plt
import datetime
import pickle
from pathlib import Path
from IPython.core.getipython import get_ipython
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19, ResNet50, VGG16, MobileNetV2, Xception, EfficientNetB0, DenseNet121
from tensorflow.keras.applications.vgg19 import preprocess_input as vgg19_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet50_preprocess
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg16_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenetv2_preprocess
from tensorflow.keras.applications.xception import preprocess_input as xception_preprocess
from tensorflow.keras.applications.efficientnet import preprocess_input as efficientnetb0_preprocess
from tensorflow.keras.applications.densenet import preprocess_input as densenet121_preprocess
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout,Dense, BatchNormalization,Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

from sklearn.metrics import confusion_matrix, roc_curve, auc, classification_report, f1_score, precision_score, recall_score


In [4]:

def split_dataset(source_dir, output_dir, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1):
    """
    Splits each image class in the source_dir into train, validation, and test directories.
    """
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    train_dir = os.path.join(output_dir, "train")
    val_dir   = os.path.join(output_dir, "validation")
    test_dir  = os.path.join(output_dir, "test")
    
    for dir_path in [train_dir, val_dir, test_dir]:
        if not os.path.exists(dir_path):
            os.makedirs(dir_path)
    # Get the list of classes, each class is a subdirectory in the source directory
    classes = [d for d in os.listdir(source_dir) if os.path.isdir(os.path.join(source_dir, d))]
    
    for class_name in classes:
        class_src = os.path.join(source_dir, class_name)
        images = [f for f in os.listdir(class_src) if os.path.isfile(os.path.join(class_src, f))]
        random.shuffle(images)
        
        total = len(images)
        train_end = int(total * train_ratio)
        val_end   = train_end + int(total * val_ratio)
        
        splits = {
            "train": images[:train_end],
            "validation": images[train_end:val_end],
            "test": images[val_end:]
        }
        
        for split, file_list in splits.items():
            class_dst = os.path.join(output_dir, split, class_name)
            if not os.path.exists(class_dst):
                os.makedirs(class_dst)
            for file_name in file_list:
                src_file = os.path.join(class_src, file_name)
                dst_file = os.path.join(class_dst, file_name)
                shutil.copy2(src_file, dst_file)
    print(f"Dataset split into train, validation, and test in '{output_dir}'.")





In [5]:

def build_model(Pretrained_model):
    x = Pretrained_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(512)(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(0.3)(x)
    predictions = Dense(1, activation='sigmoid', dtype='float32')(x)
    return Model(inputs=Pretrained_model.input, outputs=predictions)

def train_and_evaluate_model(model_name, preprocess_function):
    print(f"\nTraining with {model_name}...\n")
    
    if model_name == 'Xception':
        img_h, img_w = 299, 299
    else:
        img_h, img_w = 224, 224

    # Create ImageDataGenerators for each split
    train_datagen = ImageDataGenerator(preprocessing_function=preprocess_function)
    val_datagen   = ImageDataGenerator(preprocessing_function=preprocess_function)
    test_datagen  = ImageDataGenerator(preprocessing_function=preprocess_function)
    
    # Define paths to the split dataset directories
    train_dir = os.path.join(split_dataset_dir, "train")
    val_dir   = os.path.join(split_dataset_dir, "validation")
    test_dir  = os.path.join(split_dataset_dir, "test")
    
    # Data generators for each split
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=(img_h, img_w),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=True,
        seed=42,
    )
    
    val_generator = val_datagen.flow_from_directory(
        val_dir,
        target_size=(img_h, img_w),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False,
        seed=42,
    )
    
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=(img_h, img_w),
        batch_size=batch_size,
        class_mode='binary',
        shuffle=False  ,
    )
    
    class_indices = test_generator.class_indices    
    
    # Load the chosen pretrained model as the base
    if model_name == 'VGG19':
        Pretrained_model = VGG19(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    elif model_name == 'ResNet50':
        Pretrained_model = ResNet50(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    elif model_name == 'VGG16':
        Pretrained_model = VGG16(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    elif model_name == 'MobileNetV2':
        Pretrained_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    elif model_name == 'Xception':
        Pretrained_model = Xception(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    elif model_name == 'EfficientNetB0':
        Pretrained_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    elif model_name == 'DenseNet121':
        Pretrained_model = DenseNet121(weights='imagenet', include_top=False, input_shape=(img_h, img_w, 3))
    else:
        raise ValueError("Invalid model name")
    
    # Freeze the Pretrained_model layers
    Pretrained_model.trainable = False
    model = build_model(Pretrained_model)
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    
    early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
    
    # Train the model 
    history = model.fit(
        train_generator,
        steps_per_epoch=math.ceil(train_generator.samples / batch_size),
        epochs=epochs,
        validation_data=val_generator,
        validation_steps=math.ceil(val_generator.samples / batch_size),
        callbacks=[early_stopping],
          )
    
    # evaluate on the test set
    test_loss, test_accuracy = model.evaluate(test_generator, verbose=1)    
    print(f"\n{model_name} Test Loss: {test_loss:.4f}")
    print(f"{model_name} Test Accuracy: {test_accuracy:.4f}")
    
    # Make predictions on the test set
    predictions = model.predict(test_generator, verbose=1)
    y_pred = (predictions > 0.5).astype(int).flatten()
    y_true = test_generator.classes
    
    # Compute performance metrics
    cm = confusion_matrix(y_true, y_pred)
    print("Confusion Matrix:\n", cm)
    
    f1 = f1_score(y_true, y_pred, average='binary')
    precision = precision_score(y_true, y_pred, average='binary')
    recall = recall_score(y_true, y_pred, average='binary')
    
    print("\nClassification Report:")
    labels = list(class_indices.keys())
    print(classification_report(y_true, y_pred, target_names=labels))
    
    # Compute ROC Curve and AUC
    fpr, tpr, _ = roc_curve(y_true, predictions.flatten()) 
    roc_auc = auc(fpr, tpr)
    
    return {
        'model': model,
        'history': history.history,
        'confusion_matrix': cm,
        'roc': (fpr, tpr, roc_auc),
        'performance': {'accuracy': test_accuracy, 'f1': f1, 'precision': precision, 'recall': recall},
        'labels': labels
    }



def plot_results(results, run_dir):
    model_names = list(results.keys())
    num_models = len(model_names)
    
    # 1. Plot Confusion Matrices
    fig_cm, axes_cm = plt.subplots(1, num_models, figsize=(5*num_models, 4))
    if num_models == 1:
        axes_cm = [axes_cm]
    for ax, name in zip(axes_cm, model_names):
        cm = results[name]['confusion_matrix']
        im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        ax.set_title(f"{name} Confusion Matrix")
        tick_marks = np.arange(len(results[name]['labels']))
        ax.set_xticks(tick_marks)
        ax.set_xticklabels(results[name]['labels'], rotation=45)
        ax.set_yticks(tick_marks)
        ax.set_yticklabels(results[name]['labels'])
        thresh = cm.max() / 2.0
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                ax.text(j, i, format(cm[i, j], 'd'),
                        ha="center", va="center",
                        color="white" if cm[i, j] > thresh else "black")
        fig_cm.colorbar(im, ax=ax)
    fig_cm.tight_layout(rect=[0, 0, 1, 0.95])
    fig_cm.suptitle("Confusion Matrices", fontsize=16)
    plt.savefig(os.path.join(run_dir, "confusion_matrices.png"))
    plt.close(fig_cm)
    
    # 2. Plot ROC Curves
    fig_roc, axes_roc = plt.subplots(1, num_models, figsize=(5*num_models, 4))
    if num_models == 1:
        axes_roc = [axes_roc]
    for ax, name in zip(axes_roc, model_names):
        fpr, tpr, roc_auc = results[name]['roc']
        ax.plot(fpr, tpr, lw=2, label=f"AUC = {roc_auc:.2f}")
        ax.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
        ax.set_xlim([0.0, 1.0])
        ax.set_ylim([0.0, 1.05])
        ax.set_xlabel('False Positive Rate')
        ax.set_ylabel('True Positive Rate')
        ax.set_title(f"ROC Curve - {name}")
        ax.legend(loc="lower right")
    fig_roc.tight_layout(rect=[0, 0, 1, 0.95])
    fig_roc.suptitle("ROC Curves", fontsize=16)
    plt.savefig(os.path.join(run_dir, "roc_curves.png"))
    plt.close(fig_roc)
    
    # 3. Plot Accuracy and Loss Curves for each model
    for name in model_names:
        history = results[name]['history']
        epochs_range = range(1, len(history['accuracy']) + 1)
        fig_model, (ax_acc, ax_loss) = plt.subplots(1, 2, figsize=(12, 5))
        ax_acc.plot(epochs_range, history['accuracy'], marker='o', label='Train Accuracy')
        ax_acc.plot(epochs_range, history['val_accuracy'], marker='x', linestyle='--', label='Validation Accuracy')
        ax_acc.set_title(f"{name} Accuracy")
        ax_acc.set_xlabel('Epoch')
        ax_acc.set_ylabel('Accuracy')
        ax_acc.legend()
        
        ax_loss.plot(epochs_range, history['loss'], marker='o', label='Train Loss')
        ax_loss.plot(epochs_range, history['val_loss'], marker='x', linestyle='--', label='Validation Loss')
        ax_loss.set_title(f"{name} Loss")
        ax_loss.set_xlabel('Epoch')
        ax_loss.set_ylabel('Loss')
        ax_loss.legend()
        
        fig_model.suptitle(f"Accuracy and Loss Curves - {name}", fontsize=16)
        fig_model.tight_layout(rect=[0, 0, 1, 0.93])
        plt.savefig(os.path.join(run_dir, f"{name}_training_curves.png"))
        plt.close(fig_model)
    
    # 4. Overall Performance Comparison Table
    col_labels = ["Model", "Accuracy", "Precision", "Recall", "F1 Score"]
    cell_text = []
    for name in model_names:
        perf = results[name]['performance']
        row = [name,
               f"{perf['accuracy']:.4f}",
               f"{perf['precision']:.4f}",
               f"{perf['recall']:.4f}",
               f"{perf['f1']:.4f}"]
        cell_text.append(row)
    
    fig_table, ax_table = plt.subplots(figsize=(8, len(model_names)*0.8+1))
    ax_table.axis('tight')
    ax_table.axis('off')
    table = ax_table.table(cellText=cell_text, colLabels=col_labels, loc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1, 2)
    fig_table.suptitle("Overall Performance Comparison", fontsize=16)
    plt.savefig(os.path.join(run_dir, "performance_table.png"))
    plt.close(fig_table)
    
    # 5. Save training histories to pickle files
    for name in model_names:
        history = results[name]['history']
        with open(os.path.join(run_dir, f"{name}_history.pkl"), "wb") as f:
            pickle.dump(history, f)
    print("Training histories saved for each model.")



In [8]:

if __name__ == "__main__":
    
    ##### # Paths to dataset directory
    Base_Folder = 'D:/Learning/University of sadat/Grade 4/Semester 2/06- Graduation Project/Coding/' 
    original_dataset_dir = f'{Base_Folder}00- The DataSet/00- Dogs Femur Fracture'
    split_dataset_dir = f'{Base_Folder}00- The DataSet/Dataset_split_before_preprocess'
    ##### # Split the dataset to train, validation and testing 
    # split_dataset(original_dataset_dir, split_dataset_dir, train_ratio=0.8, val_ratio=0.1, test_ratio=0.1)
    
    current_time = datetime.datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    run_dir = os.path.join(f'{Base_Folder}runs_codes', f"before_preprocess_{current_time}")
    os.makedirs(run_dir, exist_ok=True)
    ################################ Save the Jupiter Note Book in the runs code 
    # Get the current notebook's path
    def get_notebook_path():
        ipython = get_ipython()
        if ipython is None:
            raise RuntimeError("Not running in Jupyter.")
        # VS Code specific (may not work in all environments)
        if '__vsc_ipynb_file__' in ipython.user_ns:
            return Path(ipython.user_ns['__vsc_ipynb_file__'])
        # Fallback or other environments
        try:
            return Path(ipython.startup_scripts[0])
        except:
            raise RuntimeError("Could not determine notebook path.")


    # Get the current notebook's path and name
    notebook_path = get_notebook_path()
    notebook_name = notebook_path.name
    # Create the destination path
    destination_path = os.path.join(run_dir, notebook_name)

    # Copy the notebook to the specified directory
    shutil.copy(notebook_path, destination_path)
    #########################################################################
    batch_size = 5
    epochs = 20
    
    results = {}
    models = {
        'VGG19': vgg19_preprocess,
        'ResNet50': resnet50_preprocess,
        'VGG16': vgg16_preprocess,
        'MobileNetV2': mobilenetv2_preprocess,
        'Xception': xception_preprocess,
        'EfficientNetB0': efficientnetb0_preprocess,
        'DenseNet121': densenet121_preprocess
    }
    
    for model_name, preprocess in models.items():
        results[model_name] = train_and_evaluate_model(model_name, preprocess)
        
        model_path = os.path.join(run_dir, f"{model_name}_results.pkl")
        results[model_name]['model'].save(f"{run_dir}/{model_name}_model.h5")
        print(f"{model_name} model saved.")
        plot_results(results,run_dir)
    
    print(f"All outputs saved to directory: {run_dir}")


Dataset split into train, validation, and test in 'D:/Learning/University of sadat/Grade 4/Semester 2/06- Graduation Project/Coding/00- The DataSet/Dataset_split_before_preprocess'.

Training with VGG19...

Found 41 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
Found 11 images belonging to 2 classes.
Epoch 1/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 11s 964ms/step - accuracy: 0.3704 - loss: 0.8432 - val_accuracy: 0.3333 - val_loss: 1.5608
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 939ms/step - accuracy: 0.5192 - loss: 0.7245 - val_accuracy: 0.3333 - val_loss: 1.1581
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 862ms/step - accuracy: 0.6195 - loss: 0.7991 - val_accuracy: 0.5000 - val_loss: 0.9208
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 846ms/step - accuracy: 0.6984 - loss: 0.5036 - val_accuracy: 0.6667 - val_loss: 0.8089
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 860ms/step - accuracy: 0.7505 - loss: 0.4571 - val_accuracy: 0.6667 - val_loss: 0.7167
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 8s 888m

Confusion Matrix:
 [[3 0]
 [0 8]]

Classification Report:
              precision    recall  f1-score   support

     Oblique       1.00      1.00      1.00         3
  Overriding       1.00      1.00      1.00         8

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

VGG19 model saved.
Training histories saved for each model.

Training with ResNet50...

Found 41 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
Found 11 images belonging to 2 classes.
Epoch 1/20


d:\Learning\University of sadat\Grade 4\Semester 2\06- Graduation Project\Ai_Env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 12s 669ms/step - accuracy: 0.4914 - loss: 0.8476 - val_accuracy: 0.5000 - val_loss: 0.7255
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 437ms/step - accuracy: 0.7228 - loss: 0.5925 - val_accuracy: 0.6667 - val_loss: 0.6417
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 418ms/step - accuracy: 0.7929 - loss: 0.4648 - val_accuracy: 0.8333 - val_loss: 0.5690
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 411ms/step - accuracy: 0.8763 - loss: 0.2917 - val_accuracy: 0.8333 - val_loss: 0.4991
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 421ms/step - accuracy: 0.8266 - loss: 0.3532 - val_accuracy: 0.8333 - val_loss: 0.4429
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 414ms/step - accuracy: 0.8784 - loss: 0.3239 - val_accuracy: 0.8333 - val_loss: 0.4028
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 416ms/step - accuracy: 0.8979 - loss: 0.2145 - val_accuracy: 0.8333 - val_loss: 0.3521
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 4s 447ms/step - accuracy: 0.9077 - loss: 0.3246 - val_accuracy: 1.0000 - val_loss: 0.3190
Ep

2/3 ━━━━━━━━━━━━━━━━━━━━ 0s 341ms/stepWARNING:tensorflow:6 out of the last 8 calls to <function TensorFlowTrainer.make_predict_function.<locals>.one_step_on_data_distributed at 0x0000026EADF794E0> triggered tf.function retracing. Tracing is expensive and the excessive number of tracings could be due to (1) creating @tf.function repeatedly in a loop, (2) passing tensors with different shapes, (3) passing Python objects instead of tensors. For (1), please define your @tf.function outside of the loop. For (2), @tf.function has reduce_retracing=True option that can avoid unnecessary retracing. For (3), please refer to https://www.tensorflow.org/guide/function#controlling_retracing and https://www.tensorflow.org/api_docs/python/tf/function for  more details.


...

9/9 ━━━━━━━━━━━━━━━━━━━━ 14s 548ms/step - accuracy: 0.5203 - loss: 0.6848 - val_accuracy: 0.6667 - val_loss: 0.6442
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 200ms/step - accuracy: 0.6129 - loss: 0.7716 - val_accuracy: 0.8333 - val_loss: 0.6071
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 211ms/step - accuracy: 0.7012 - loss: 0.5102 - val_accuracy: 0.8333 - val_loss: 0.5776
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 199ms/step - accuracy: 0.6874 - loss: 0.5420 - val_accuracy: 1.0000 - val_loss: 0.5446
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 193ms/step - accuracy: 0.7163 - loss: 0.5606 - val_accuracy: 1.0000 - val_loss: 0.5080
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 195ms/step - accuracy: 0.8707 - loss: 0.3940 - val_accuracy: 1.0000 - val_loss: 0.4728
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 190ms/step - accuracy: 0.6815 - loss: 0.4844 - val_accuracy: 1.0000 - val_loss: 0.4342
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 212ms/step - accuracy: 0.8625 - loss: 0.4078 - val_accuracy: 1.0000 - val_loss: 0.4020
Ep

Confusion Matrix:
 [[3 0]
 [0 8]]

Classification Report:
              precision    recall  f1-score   support

     Oblique       1.00      1.00      1.00         3
  Overriding       1.00      1.00      1.00         8

    accuracy                           1.00        11
   macro avg       1.00      1.00      1.00        11
weighted avg       1.00      1.00      1.00        11

EfficientNetB0 model saved.
Training histories saved for each model.

Training with DenseNet121...

Found 41 images belonging to 2 classes.
Found 6 images belonging to 2 classes.
Found 11 images belonging to 2 classes.
Epoch 1/20


d:\Learning\University of sadat\Grade 4\Semester 2\06- Graduation Project\Ai_Env\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 18s 850ms/step - accuracy: 0.6248 - loss: 0.7300 - val_accuracy: 0.3333 - val_loss: 0.7093
Epoch 2/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 380ms/step - accuracy: 0.7259 - loss: 0.5800 - val_accuracy: 0.5000 - val_loss: 0.6656
Epoch 3/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 372ms/step - accuracy: 0.5412 - loss: 0.6868 - val_accuracy: 0.5000 - val_loss: 0.6240
Epoch 4/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 377ms/step - accuracy: 0.6651 - loss: 0.5226 - val_accuracy: 0.5000 - val_loss: 0.5915
Epoch 5/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 371ms/step - accuracy: 0.7340 - loss: 0.4768 - val_accuracy: 0.6667 - val_loss: 0.5616
Epoch 6/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 372ms/step - accuracy: 0.9133 - loss: 0.3528 - val_accuracy: 0.6667 - val_loss: 0.5252
Epoch 7/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 372ms/step - accuracy: 0.8177 - loss: 0.3831 - val_accuracy: 0.6667 - val_loss: 0.4890
Epoch 8/20
9/9 ━━━━━━━━━━━━━━━━━━━━ 3s 369ms/step - accuracy: 0.9014 - loss: 0.2879 - val_accuracy: 0.6667 - val_loss: 0.4666
Ep

Confusion Matrix:
 [[3 0]
 [1 7]]

Classification Report:
              precision    recall  f1-score   support

     Oblique       0.75      1.00      0.86         3
  Overriding       1.00      0.88      0.93         8

    accuracy                           0.91        11
   macro avg       0.88      0.94      0.90        11
weighted avg       0.93      0.91      0.91        11

DenseNet121 model saved.
Training histories saved for each model.
All outputs saved to directory: D:/Learning/University of sadat/Grade 4/Semester 2/06- Graduation Project/Coding/runs_codes\before_preprocess_2025-04-21_00-29-45
